In [43]:
import pandas as pd

def extract_unique_users(phase, previous_phase=None, user_id_column='user_id'):
    """
    从CSV文件中提取不重复的user_id并保存到新文件
    
    Parameters:
    phase: 比赛阶段
    previous_phase: 前一个比赛阶段
    user_id_column: user_id列名
    """
    input_file = f'csv_data/inter_{phase}.csv'
    previous_phase_file = f'csv_data/inter_{previous_phase}.csv' if previous_phase else None
    output_file = f'csv_data/users_{phase}_only.csv'
    try:
        # 读取数据
        df = pd.read_csv(input_file)
        previous_phase_df = None
        if previous_phase:
            previous_phase_df = pd.read_csv(previous_phase_file)
        # 提取不重复的user_id
        unique_users = df[user_id_column].drop_duplicates().reset_index(drop=True)
        unique_users = unique_users[~unique_users.isin(previous_phase_df[user_id_column])] if previous_phase else unique_users
        # 创建新DataFrame
        result_df = pd.DataFrame({user_id_column: unique_users})
        
        # 保存到新文件
        result_df.to_csv(output_file, index=False)
        
        print(f"成功提取 {len(unique_users)} 个不重复用户")
        print(f"结果已保存到: {output_file}")
        
        return result_df
        
    except Exception as e:
        print(f"处理过程中出错: {e}")
        return None


In [44]:
phase = ['preliminary', 'reevaluation', 'final']
for i in range(len(phase)):
    extract_unique_users(phase[i],phase[i-1]) if i > 0 else extract_unique_users(phase[i],None)

成功提取 600 个不重复用户
结果已保存到: csv_data/users_preliminary_only.csv
成功提取 400 个不重复用户
结果已保存到: csv_data/users_reevaluation_only.csv
成功提取 451 个不重复用户
结果已保存到: csv_data/users_final_only.csv


In [45]:
import os
from datetime import datetime

def process_user_interaction_from_csv(phase):
    
    map_dict = {
        'pre': 'preliminary',
        'semi': 'reevaluation',
        'final': 'final'
    }
    # 文件路径配置 - 请根据你的实际情况修改
    file_paths = {
        'table_a': f'csv_data/users_{map_dict[phase]}_only.csv',  # 用户表
        'table_b': 'csv_data/inter_preliminary.csv',  # 交互数据表B
        'table_c': 'csv_data/inter_reevaluation.csv',  # 交互数据表C
        'table_d': 'csv_data/inter_final.csv'   # 交互数据表D
    }

    # 输出文件路径
    output_file = f'csv_data/interaction_{phase}_only.csv'
    
    try:
        # 1. 从表A读取user_id
        print("正在从表A读取用户ID...")
        users_df = pd.read_csv(file_paths['table_a'])
        
        # 检查user_id列是否存在
        if 'user_id' not in users_df.columns:
            # 尝试常见的用户ID列名
            user_id_columns = ['user_id', 'userid', 'userId', 'UserID', '用户ID', '用户编号']
            user_id_col = None
            for col in user_id_columns:
                if col in users_df.columns:
                    user_id_col = col
                    break
            
            if user_id_col is None:
                raise ValueError("未找到用户ID列，请检查CSV文件列名")
        else:
            user_id_col = 'user_id'
        
        user_ids = users_df[user_id_col].tolist()
        print(f"共读取到 {len(user_ids)} 个用户ID")
        
        # 2. 从表B、C、D读取交互数据
        print("正在从表B、C、D读取交互数据...")
        
        all_interactions = []
        
        for table_key in ['table_b', 'table_c', 'table_d']:
            print(f"读取 {table_key}...")
            df = pd.read_csv(file_paths[table_key])
            
            # 查找用户ID列
            user_id_col_interaction = None
            for col in ['user_id', 'userid', 'userId', 'UserID', '用户ID', '用户编号']:
                if col in df.columns:
                    user_id_col_interaction = col
                    break
            
            if user_id_col_interaction is None:
                print(f"警告: {table_key} 中未找到用户ID列，跳过该表")
                continue
            
            # 查找借阅时间列
            borrow_time_col = None
            for col in ['borrow_time', 'borrowtime', 'borrowTime', '借阅时间', '时间', 'timestamp']:
                if col in df.columns:
                    borrow_time_col = col
                    break
            
            # 筛选匹配的用户数据
            filtered_df = df[df[user_id_col_interaction].isin(user_ids)].copy()
            filtered_df['source_table'] = table_key  # 添加来源表标记
            
            # 重命名列以便统一处理
            if user_id_col_interaction != 'user_id':
                filtered_df.rename(columns={user_id_col_interaction: 'user_id'}, inplace=True)
            
            if borrow_time_col and borrow_time_col != 'borrow_time':
                filtered_df.rename(columns={borrow_time_col: 'borrow_time'}, inplace=True)
            
            all_interactions.append(filtered_df)
            print(f"{table_key} 匹配到 {len(filtered_df)} 条记录")
        
        # 3. 合并数据
        print("正在合并数据...")
        if not all_interactions:
            raise ValueError("没有找到任何匹配的交互数据")
        
        combined_df = pd.concat(all_interactions, ignore_index=True)
        print(f"合并后总数据量: {len(combined_df)} 条")
        
        # 4. 按照借阅时间排序并去重
        print("正在按借阅时间排序并去重...")
        
        # 检查是否有借阅时间列
        if 'borrow_time' in combined_df.columns:
            # 转换借阅时间为datetime格式
            combined_df['borrow_time'] = pd.to_datetime(
                combined_df['borrow_time'], 
                errors='coerce'  # 转换失败设为NaT
            )
            
            # 按用户ID和借阅时间排序
            combined_df_sorted = combined_df.sort_values(['user_id', 'borrow_time'])
            
            # 去重 - 根据用户ID和借阅时间完全去重
            deduplicated_df = combined_df_sorted.drop_duplicates(
                subset=['user_id', 'borrow_time'],  
                keep='first'  # 保留第一条记录
            )
        else:
            # 如果没有借阅时间列，只按用户ID去重
            print("警告: 未找到借阅时间列，仅按用户ID去重")
            combined_df_sorted = combined_df.sort_values('user_id')
            deduplicated_df = combined_df_sorted.drop_duplicates(
                subset=['user_id'],  
                keep='first'
            )
        
        print(f"去重后数据量: {len(deduplicated_df)} 条")
        
        # 5. 写入新CSV文件
        print("正在写入新CSV文件...")
        deduplicated_df.to_csv(output_file, index=False, encoding='utf-8-sig')
        
        print(f"数据已成功写入文件: {output_file}")
        print(f"最终数据统计:")
        print(f"- 总用户数: {deduplicated_df['user_id'].nunique()}")
        print(f"- 总记录数: {len(deduplicated_df)}")
        
        if 'borrow_time' in deduplicated_df.columns:
            valid_times = deduplicated_df['borrow_time'].dropna()
            if len(valid_times) > 0:
                print(f"- 时间范围: {valid_times.min()} 到 {valid_times.max()}")
        
        return deduplicated_df
        
    except Exception as e:
        print(f"处理过程中发生错误: {e}")
        return None

In [46]:
phases = ['pre', 'semi', 'final']
for phase in phases:
    process_user_interaction_from_csv(phase)

正在从表A读取用户ID...
共读取到 600 个用户ID
正在从表B、C、D读取交互数据...
读取 table_b...
table_b 匹配到 35833 条记录
读取 table_c...
table_c 匹配到 35881 条记录
读取 table_d...
table_d 匹配到 35869 条记录
正在合并数据...
合并后总数据量: 107583 条
正在按借阅时间排序并去重...
去重后数据量: 44186 条
正在写入新CSV文件...
数据已成功写入文件: csv_data/interaction_pre_only.csv
最终数据统计:
- 总用户数: 600
- 总记录数: 44186
- 时间范围: 2020-01-02 11:16:34 到 2024-12-30 16:37:44
正在从表A读取用户ID...
共读取到 400 个用户ID
正在从表B、C、D读取交互数据...
读取 table_b...
table_b 匹配到 0 条记录
读取 table_c...
table_c 匹配到 22446 条记录
读取 table_d...
table_d 匹配到 22513 条记录
正在合并数据...
合并后总数据量: 44959 条
正在按借阅时间排序并去重...
去重后数据量: 26864 条
正在写入新CSV文件...
数据已成功写入文件: csv_data/interaction_semi_only.csv
最终数据统计:
- 总用户数: 400
- 总记录数: 26864
- 时间范围: 2020-01-02 10:13:48 到 2024-12-30 14:38:15
正在从表A读取用户ID...
共读取到 451 个用户ID
正在从表B、C、D读取交互数据...
读取 table_b...
table_b 匹配到 0 条记录
读取 table_c...
table_c 匹配到 0 条记录
读取 table_d...
table_d 匹配到 23928 条记录
正在合并数据...
合并后总数据量: 23928 条
正在按借阅时间排序并去重...
去重后数据量: 23858 条
正在写入新CSV文件...
数据已成功写入文件: csv_data/interaction_final_only.csv
最终数据统计:
- 总用户数:

In [47]:
import pandas as pd
import numpy as np
import os
import time

def process_data_to_recbole(phase):

    DATA_DIR = f'./dataset/library_data_{phase}'
    os.makedirs(DATA_DIR, exist_ok=True)
    print(f"数据输出目录已创建/检查: {DATA_DIR}")

    FILE_PREFIX = f'library_data_{phase}'

    try:
        df_book = pd.read_csv('./csv_data/item.csv')
        df_inter = pd.read_csv(f'./csv_data/interaction_{phase}_only.csv')
        df_user = pd.read_csv('./csv_data/user.csv')
        print("原始文件加载成功。")
    except FileNotFoundError as e:
        print(f"错误：未能找到文件，请检查路径。缺失文件: {e}")
    
    print("\n--- 3. 正在处理交互文件 (inter.csv) ---")
    
    inter_data = df_inter[['user_id', 'book_id', 'borrow_time']].copy()
    
    inter_data.rename(columns={
        'user_id': 'user_id:token',
        'book_id': 'item_id:token',
        'borrow_time': 'timestamp:float'
    }, inplace=True)
    
    inter_data['timestamp:float'] = inter_data['timestamp:float'].apply(
        lambda x: time.mktime(time.strptime(str(x), '%Y-%m-%d %H:%M:%S'))
    )
    
    inter_data['label:float'] = 1.0
    
    # === 添加打乱操作 ===
    print("正在打乱交互数据...")
    inter_data = inter_data.sample(frac=1, random_state=42).reset_index(drop=True)
    print(f"打乱完成，共 {len(inter_data)} 条交互记录")
    
    inter_output_path = os.path.join(DATA_DIR, f'{FILE_PREFIX}.inter')
    inter_data.to_csv(inter_output_path, sep='\t', index=False)
    print(f"交互文件已保存至: {inter_output_path}")
    
    # --- 4. 处理 BOOK.CSV -> library_data.item ---
    
    print("\n--- 4. 正在处理物品文件 (book.csv) ---")
    
    item_data = df_book.copy()
    
    item_data.rename(columns={
        'book_id': 'item_id:token',
        '题名': 'title:token',
        '作者': 'author:token',
        '出版社': 'publisher:token',
        '一级分类': 'category_l1:token',
        '二级分类': 'category_l2:token'
    }, inplace=True)
    
    item_output_path = os.path.join(DATA_DIR, f'{FILE_PREFIX}.item')
    item_data.to_csv(item_output_path, sep='\t', index=False)
    print(f"物品文件已保存至: {item_output_path}")
    
    # --- 5. 处理 USER.CSV -> library_data.user ---
    
    print("\n--- 5. 正在处理用户文件 (user.csv) ---")
    
    active_user_ids = set(df_inter['user_id'].unique())
    print(f"交互数据中共有 {len(active_user_ids)} 个活跃用户")
    
    # user_data = df_user[df_user['借阅人'].isin(active_user_ids)].copy()
    user_data = df_user.copy()
    print(f"过滤后用户数据包含 {len(user_data)} 个用户")
    
    # 5.3 核心列重命名和标记
    user_data.rename(columns={
        '借阅人': 'user_id:token',
        '性别': 'gender:token',
        'DEPT': 'dept:token',
        '年级': 'grade:token',
        '类型': 'user_type:token'
    }, inplace=True)
    
    user_output_path = os.path.join(DATA_DIR, f'{FILE_PREFIX}.user')
    user_data.to_csv(user_output_path, sep='\t', index=False)
    print(f"用户文件已保存至: {user_output_path}")
    
    print("\n--- 6. 数据一致性检查 ---")
    
    inter_user_ids = set(inter_data['user_id:token'])
    user_file_ids = set(user_data['user_id:token'])
    
    missing_in_user = inter_user_ids - user_file_ids
    extra_in_user = user_file_ids - inter_user_ids
    
    print(f"交互文件中的用户数量: {len(inter_user_ids)}")
    print(f"用户文件中的用户数量: {len(user_file_ids)}")
    print(f"只在交互文件中出现的用户: {len(missing_in_user)}")
    print(f"只在用户文件中出现的用户: {len(extra_in_user)}")
    
    if missing_in_user:
        print(f"警告：以下用户出现在交互数据但不在用户数据中: {list(missing_in_user)[:5]}...")  # 只显示前5个
    if extra_in_user:
        print(f"警告：以下用户出现在用户数据但不在交互数据中: {list(extra_in_user)[:5]}...")  # 只显示前5个
    
    print("\n==================================")
    print("RecBole 数据文件生成完毕！")
    print("您已经成功创建了以下文件，可直接用于 RecBole 训练：")
    print(f"- {inter_output_path}")
    print(f"- {item_output_path}")
    print(f"- {user_output_path}")
    print("==================================")

In [48]:
for phase in phases:
    process_data_to_recbole(phase)

数据输出目录已创建/检查: ./dataset/library_data_pre
原始文件加载成功。

--- 3. 正在处理交互文件 (inter.csv) ---
正在打乱交互数据...
打乱完成，共 44186 条交互记录
交互文件已保存至: ./dataset/library_data_pre/library_data_pre.inter

--- 4. 正在处理物品文件 (book.csv) ---
物品文件已保存至: ./dataset/library_data_pre/library_data_pre.item

--- 5. 正在处理用户文件 (user.csv) ---
交互数据中共有 600 个活跃用户
过滤后用户数据包含 1451 个用户
用户文件已保存至: ./dataset/library_data_pre/library_data_pre.user

--- 6. 数据一致性检查 ---
交互文件中的用户数量: 600
用户文件中的用户数量: 1451
只在交互文件中出现的用户: 0
只在用户文件中出现的用户: 851
警告：以下用户出现在用户数据但不在交互数据中: [3, 5, 7, 12, 13]...

RecBole 数据文件生成完毕！
您已经成功创建了以下文件，可直接用于 RecBole 训练：
- ./dataset/library_data_pre/library_data_pre.inter
- ./dataset/library_data_pre/library_data_pre.item
- ./dataset/library_data_pre/library_data_pre.user
数据输出目录已创建/检查: ./dataset/library_data_semi
原始文件加载成功。

--- 3. 正在处理交互文件 (inter.csv) ---
正在打乱交互数据...
打乱完成，共 26864 条交互记录
交互文件已保存至: ./dataset/library_data_semi/library_data_semi.inter

--- 4. 正在处理物品文件 (book.csv) ---
物品文件已保存至: ./dataset/library_data_semi/library_data_semi.item